# Assignment 2: Decision Trees

In this assignment, you are going to implement a decision tree (or random forest) to forcast the weather.

## Description

- You must implement a `Model` class for training and prediction:
  ```python
  X, y = load_dataset()

  model = Model(num_features, num_classes)

  # training
  model.fit(X, y)

  # prediction
  y_pred = model.predict(X)
  ```
- Please search (Ctrl+F) for `TODO` to see what you need to do. You have to implement the classifier from scratch (do not directly use the classifier in scikit-learn).
- About the dataset
  - Given the **training set**, please **train/validate** on it.  
  (note that your model will get bad testing score if it overfits on the training set)
  - After submitting the assignment, we will train on the same training set and test on the hidden **testing set** for scoring (using [f1-score](https://towardsdatascience.com/a-look-at-precision-recall-and-f1-score-36b5fd0dd3ec#11b8)).

### Typical performance

- **Random Guess**  
  F1-Score: 0.30  
  Accuracy: 0.50
- **Always Predict 1**  
  F1-Score: 0.37  
  Accuracy: 0.22
- **Always Predict 0**  
  F1-Score: 0.00  
  Accuracy: 0.77
- **sklearn.tree.DecisionTreeClassifier**  
  - **Training (5-fold cross-validation mean)**  
    F1-Score: 0.63-0.99  
    Accuracy: 0.85-0.99
  - **Validation (5-fold cross-validation mean)**  
    F1-Score: 0.50-0.60  
    Accuracy: 0.75-0.90


In [10]:
###########################
# DO NOT CHANGE THIS CELL #
###########################

import os
import urllib.request
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score


def load_dataset(url):
  """ Get and load weather dataset. """

  path = url.split('/')[-1] # get the file name from url

  if not os.path.exists(path):
    print('Download:', url)
    urllib.request.urlretrieve(url, path)

  return pd.read_pickle(path) # pickle protocol=4


def get_input_target(df):
  """ Get X and y from weather dataset. """
  
  target_column = 'RainTomorrow' # predict 1 of it rains tomorrow

  X = df.drop(columns=[target_column]).to_numpy()
  y = df[target_column].to_numpy()

  return X, y


def k_fold_cv(model_create_fn, X, y, k=5):
  """ Run k-fold cross-validation. """

  results = []

  idxs = list(range(X.shape[0]))
  np.random.shuffle(idxs)

  for i, (train_idxs, val_idxs) in enumerate(KFold(k).split(idxs)):
    splits = {'train': (X[train_idxs], y[train_idxs]),
              'val':   (X[val_idxs],   y[val_idxs]  )}

    print('Run {}:'.format(i+1))

    model = model_create_fn()
    model.fit(*splits['train']) # training

    for name, (X_split, y_split) in splits.items():
      y_pred = model.predict(X_split)

      result = {'split': name,
                'f1': f1_score(y_pred, y_split),
                'acc': accuracy_score(y_pred, y_split)}
      results.append(result)

      print('{split:>8s}: f1={f1:.4f} acc={acc:.4f}'.format(**result))

  return pd.DataFrame(results)


# @begin

In [8]:
# TODO: you can define or import something here (optional)
from __future__ import division
import random
import numpy as np
from scipy.stats import mode
from collections import Counter
import time



class Model:

  def __init__(self, num_features, num_classes: int):
    """
    Initialize the model.

    Args:
        num_features (int) : the input feature size.
        num_classes (int) : number of output classes.
    """

    self.num_features = num_features
    self.num_classes = num_classes
    self.max_depth=10
    # TODO: implement your model initialization here (optional)


  def build_tree(self,X,y,feature_indexes,depth):
    if len(y) < self.num_classes or entropy(y) is 0 or depth is self.max_depth:
      
      return mode(y)[0][0]
    feature_index,threshold=find_split(X,y,feature_indexes)
    X_true,y_true,X_false,y_false=split(X,y,feature_index,threshold)
    if y_false.shape[0] is 0 or y_true.shape[0] is 0:
      return mode(y)[0][0]
    
    branch_true=self.build_tree(X_true,y_true,feature_indexes,depth+1)
    branch_false=self.build_tree(X_false,y_false,feature_indexes,depth+1)
    return node(feature_index,threshold,branch_true,branch_false)
  def fit(self, X: np.ndarray, y: np.ndarray):
    """
    Train on input/target pairs.

    Args:
        X (np.ndarray) : training inputs with shape (num_inputs, num_features).
        y (np.ndarray) : training targets with shape (num_inputs,).
    """

    # TODO: implement your training algorithm here
    n_features=X.shape[1]
    n_sub_features=(self.num_features)
    feature_indexes=random.sample(range(n_features),n_sub_features)
    self.tree=self.build_tree(X,y,feature_indexes,0)
    
  def predict(self, X: np.ndarray) -> np.ndarray:
    '''
    Predict y given X.

    Args:
        X (np.ndarray) : inputs, shape: (num_inputs, num_features).
    
    Returns:
        np.ndarray : the predicted integer outputs, shape: (num_inputs,).
    '''

    # TODO: implement your prediction algorithm here
    #p = np.random.randint(0, self.num_classes, size=X.shape[0]) # (delete this)
    num_sample=X.shape[0]
    p=np.empty(num_sample)
    for i in range(num_sample):
      n=self.tree
      while isinstance(n,node):
        if X[i][n.feature_index] <= n.threshold:
          n=n.branch_true
        else:
          n=n.branch_false
      p[i]=n
    return p


    
class node(object):
  def __init__(self,feature_index,threshold,branch_true,branch_false):
    self.feature_index=feature_index
    self.threshold=threshold
    self.branch_true=branch_true
    self.branch_false=branch_false    

def find_split(X,y,feature_indexes):
  num_features=X.shape[1]
  best_gain=0
  best_feature_index=0
  best_threshold=0
  for feature_index in feature_indexes:
    values=sorted(set(X[:,feature_index]))
    total=0
    for i in range(len(values)):
      total+=values[i]
    for i in range(7):
      pivot=int(random.uniform(0,len(values)-1))
      threshold=values[pivot]
      #total=total*random.uniform(0.1,0.5)
      X_true,y_true,X_false,y_false=split(X,y,feature_index,threshold)
      gain=information_gain(y,y_true,y_false)
      if gain > best_gain:
        best_gain=gain
        best_feature_index=feature_index
        best_threshold=threshold
  return  best_feature_index,best_threshold

def split(X,y,feature_index,threshold):
  X_true=[]
  y_true=[]
  X_false=[]
  y_false=[]
  for i in range(len(y)):
    if X[i][feature_index] <= threshold :
      X_true.append(X[i])
      y_true.append(y[i])
    else:
      X_false.append(X[i])      
      y_false.append(y[i])
  X_true=np.array(X_true)
  y_true=np.array(y_true)
  X_false=np.array(X_false)
  y_false=np.array(y_false)

  return X_true,y_true,X_false,y_false

def entropy(Y):
  start=time.process_time()
  
  distribution=Counter(Y)
  s=0.0
  total=len(Y)
  for y,num_y in distribution.items():
    p_y=(num_y/total)
    s+=p_y*np.log(p_y)
  return -s  

def information_gain(y,y_true,y_false):
  return entropy(y)-(entropy(y_true)*len(y_true)+entropy(y_false)*len(y_false))/len(y)



  # TODO: define your methods if needed (optional)

In [11]:
# @end

###########################
# DO NOT CHANGE THIS CELL #
###########################

df = load_dataset('https://lab.djosix.com/weather.pkl')
X_train, y_train = get_input_target(df)

df.head(100000)

Download: https://lab.djosix.com/weather.pkl


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,Temp9am,RainToday,RainTomorrow
0,7.400000,25.100000,0.0,5.469824,7.624853,44.0,4.0,22.0,44.0,25.0,1010.599976,4.43719,4.503167,17.200001,0.0,0
1,12.900000,25.700001,0.0,5.469824,7.624853,46.0,19.0,26.0,38.0,30.0,1007.599976,4.43719,2.000000,21.000000,0.0,0
2,9.200000,28.000000,0.0,5.469824,7.624853,24.0,11.0,9.0,45.0,16.0,1017.599976,4.43719,4.503167,18.100000,0.0,0
3,17.500000,32.299999,1.0,5.469824,7.624853,41.0,7.0,20.0,82.0,33.0,1010.799988,7.00000,8.000000,17.799999,0.0,0
4,14.600000,29.700001,0.2,5.469824,7.624853,56.0,19.0,24.0,55.0,23.0,1009.200012,4.43719,4.503167,20.600000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,20.200001,43.099998,0.0,5.469824,12.500000,43.0,20.0,13.0,16.0,10.0,1015.799988,0.00000,4.503167,34.099998,0.0,0
99996,20.799999,44.299999,0.0,5.469824,12.600000,39.0,13.0,26.0,15.0,10.0,1014.799988,0.00000,4.503167,35.799999,0.0,0
99997,16.900000,45.099998,0.0,5.469824,12.500000,39.0,4.0,19.0,35.0,15.0,1013.700012,0.00000,4.503167,30.100000,0.0,0
99998,15.900000,38.099998,0.0,5.469824,12.300000,37.0,4.0,22.0,34.0,30.0,1013.299988,4.43719,4.503167,27.100000,0.0,0


In [12]:
###########################
# DO NOT CHANGE THIS CELL #
###########################

create_model = lambda: Model(X_train.shape[1], 2)
k_fold_cv(create_model, X_train, y_train).groupby('split').mean()

Run 1:
   train: f1=0.6157 acc=0.8534
     val: f1=0.5311 acc=0.8341
Run 2:
   train: f1=0.6066 acc=0.8553
     val: f1=0.5514 acc=0.8180
Run 3:
   train: f1=0.6106 acc=0.8545
     val: f1=0.5710 acc=0.8240
Run 4:
   train: f1=0.6043 acc=0.8527
     val: f1=0.5544 acc=0.8271
Run 5:
   train: f1=0.6003 acc=0.8450
     val: f1=0.5517 acc=0.8526


,f1,acc
split,,
train,0.607497,0.852158
val,0.551914,0.831191


## Submission

1. Make sure your code runs correctly after clicking `"Runtime" > "Restart and run all"`
2. Rename this notebook to `XXXXXXX_2.ipynb`, where `XXXXXXX` is your student ID.
3. Download IPython notebook: `"File" > "Download" > "Download .ipynb"`
4. Download Python source code: `"File" > "Download" > "Download .py"`
5. Create a zip file for `XXXXXXX_2.ipynb` and `XXXXXXX_2.py`  
   named `XXXXXXX_2.zip`, where `XXXXXXX` is your student ID.
6. Upload the zip file to E3.

😊 Good luck!